## 1.1 Import Python Libraries

In [ ]:
import os, sagemaker, subprocess, boto3
from datetime import datetime
from sagemaker import s3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer

In [ ]:
!pip3 install ultralytics
from ultralytics import YOLO

# Define models
models = ["segmentation-model.pt", "slight-improved-classify.pt"]

## 1.3 Zip the code and model into `model.tar.gz` and upload it to specific S3 bucket
Here permission is granted to the S3 bucket created with CDK and not any other bucket

In [ ]:
s3_client = boto3.client('s3')
response = s3_client.list_buckets()

bucket = None
for b in response['Buckets']:
    if 'yolov11' in b["Name"]:
        bucket = 's3://' + b["Name"]
        break

if not bucket:
    raise ValueError("No suitable S3 bucket found.")

print(f'Bucket: {bucket}')
sess = sagemaker.Session(default_bucket=bucket.split('s3://')[-1])

prefix = "yolov11/dripdrop-ai-endpoint"

In [ ]:
# Upload both models
model_data_paths = {}

for model_name in models:
    # Archive model
    bashCommand = f"tar -cpzf {model_name}.tar.gz {model_name} code/"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    process.communicate()

    # Upload to S3
    model_data = s3.S3Uploader.upload(f"{model_name}.tar.gz", f"{bucket}/{prefix}")
    model_data_paths[model_name] = model_data
    print(f'Uploaded {model_name}: {model_data}')

In [ ]:
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = get_execution_role()
print(f'Role: {role}')

models_deployed = {}

## 1.4 Create the SageMaker PyTorchModel

In [ ]:
for model_name, model_data in model_data_paths.items():
    pytorch_model = PyTorchModel(
        entry_point='inference.py',
        model_data=model_data,
        framework_version='1.12',
        py_version='py38',
        role=role,
        env={'TS_MAX_RESPONSE_SIZE': '20000000', 'YOLOV11_MODEL': model_name},
        sagemaker_session=sess
    )
    
    # %% [markdown]
    # ## 1.5 Deploy Each Model on Separate SageMaker Endpoints

    INSTANCE_TYPE = 'ml.m5.4xlarge'
    ENDPOINT_NAME = f'yolov11-{model_name.replace(".pt", "")}-' + datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f')

    # Store the endpoint name in history to be accessed by other notebooks
    %store ENDPOINT_NAME
    print(f'Deploying {model_name} to Endpoint: {ENDPOINT_NAME}')

    predictor = pytorch_model.deploy(
        initial_instance_count=1, 
        instance_type=INSTANCE_TYPE,
        deserializer=JSONDeserializer(),
        endpoint_name=ENDPOINT_NAME
    )

    models_deployed[model_name] = predictor

print("Deployment completed. Endpoints created for both models.")